In [0]:
#https://www.kaggle.com/uciml/mushroom-classification
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('mushrooms.csv')
df.head()

class cap-shape cap-surface  ... spore-print-color population habitat
0     p         x           s  ...                 k          s       u
1     e         x           s  ...                 n          n       g
2     e         b           s  ...                 n          n       m
3     p         x           y  ...                 k          s       u
4     e         x           s  ...                 n          a       g

[5 rows x 23 columns]

My note: All values are categorical so NB should work well as probabilities can be easily caluculated. Intersting

In [5]:
df.shape

(8124, 23)

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
le = LabelEncoder()
de = df.apply(le.fit_transform)

type(de)

pandas.core.frame.DataFrame

In [8]:
de.head()

class  cap-shape  cap-surface  ...  spore-print-color  population  habitat
0      1          5            2  ...                  2           3        5
1      0          5            2  ...                  3           2        1
2      0          0            2  ...                  3           2        3
3      1          5            3  ...                  2           3        5
4      0          5            2  ...                  3           0        1

[5 rows x 23 columns]

My note: NB takes probability(of occurance) into account so actual value will not have any effect(no need for one-hot encoding, but conversion to no. is done to use it has numpy array)

In [9]:
data = de.values
data.shape

(8124, 23)

In [10]:
type(data)

numpy.ndarray

In [12]:
print(data[:5, :]) #.head()

[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


In [13]:
X = data[:, 1:]
y = data[:, 0]
print(X.shape)
y.shape

(8124, 22)


(8124,)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6499, 22)
(1625, 22)
(6499,)
(1625,)


In [15]:
np.unique(y_train)

array([0, 1])

0 => e => Edible </br>
1 => p => Poisonous



In [0]:
# or we can use le.inverse_transfor(y)

### Now naive bayes classifier

![alt text](https://www.codespeedy.com/wp-content/uploads/2019/06/Formula-of-Naive-Bayes-theory.png)


![alt text](https://pbiswas101.files.wordpress.com/2018/10/conditional-probability.png?w=450)

We don't need p(x) as it will be common 

My note: Images help

In [0]:
def prior_prob(y_train, label):
  total = y_train.shape[0]
  class_ex = np.sum(y_train == label)
  return class_ex/float(total)

In [0]:
def cond_prob(x_train, y_train, feature_col, feature_val, label):
  x_filtered = x_train[y_train == label]
  
  num = np.sum(x_filtered[:, feature_col] == feature_val)
  dem = np.sum(y_train == label)
  
  return num/float(dem)

In [0]:
def predict(x_train, y_train, x_test):
  classes = np.unique(y_train)
  n_features = x_train.shape[1]
  posterior_probs = []
  
  for label in classes:
    likelihood = 1.0
    for f in range(n_features):
      conditional = cond_prob(x_train, y_train, f, x_test[f], label)
      likelihood *= conditional
      
    prior = prior_prob(y_train, label)
    post = likelihood * prior
    
    posterior_probs.append(post)
    
  return np.argmax(posterior_probs)

In [0]:
# can also use inverse transfor in label encoder
def give_me_label(pred):
  if pred == 0:
    return 'Edible'
  elif pred == 1:
    return 'Poisonous'
  else:
    return 'Whaaaat!'

now time for predictions

In [22]:
print(X_test[1])

[5 3 4 1 5 1 0 0 7 1 1 2 2 7 6 0 2 1 4 2 4 0]


In [30]:
give_me_label(predict(X_train, y_train, X_test[1]))

'Edible'

In [31]:
give_me_label(y_train[1])

'Edible'

It works

In [0]:
#Functions are definitly not very efficient ut it works hehe 😅
def accuracy(x_train, y_train, x_test, y_test):
  preds = []
  for i in range(x_test.shape[0]):
    pred = predict(x_train, y_train, x_test[i])
    preds.append(pred)
    
  preds = np.array(preds)
  acc = np.sum(preds == y_test)/float(y_test.shape[0])
  return acc

In [34]:
accuracy(X_train, y_train, X_test, y_test)

0.9956923076923077

Wow! what the accuracy